In [ ]:
import EEGReptile as er   # import Reptile-EEG lib
import copy

### Prepare data and saving folders for experiment

In [ ]:
experiment_name = er.experiment_storage(experiment_name='BCI_IV_EEGNet_example')   #create storage for models and new experiment data
dataset = er.MetaDataset(dataset_name='BCI_IV')   #load dataset created in load data example
print('Dataset ' + dataset.dataset_name + ' loaded.' + ' subjects: ' + str(dataset.subjects))
targ = [7] # choose target subject to perform tests (Zero-. Few-shot) later
tr_sub = copy.deepcopy(dataset.subjects)
for sub in targ:   # remove target subject from 
    tr_sub.remove(sub) 

### Prepare the model

In [ ]:
mparams = {   # params of EEGNet model
    "model_type": "inEEG_net",
    "num_classes": 4,
    "dropout1": 0.52,
    "dropout2": 0.36,
    "f1": 16,
    "sampling_rate": 250,
    "num_channels": 22,
    "depth_multiplier": 12,
    "time_of_interest": 500,
    "time_points": 625,
    "lowpass": 50,
    "point_reducer": 5
}
model = er.model_from_params(mparams) # crate model with specified params

### Find the meta-learning hyperparameters
`
meta_params(metadataset: MetaDataset, model, tr_sub: list, tst_sub, trials, jobs, mode='single_batch',
                double_meta_step=False, meta_optimizer=False, experiment_name='experiment')
` 
meta_params - function used for meta hyper-params search
- **metadataset**: this is working dataset
- **tr_sub**: list of subjects used for training in params search
- **tst_sub**: list of subjects or None used for testing ACC in params search if none 2-k fold of tr_sub is used
- **model**: model for which params search will be performed
- **trials**: number of optuna trials in param search
- **jobs**: how many parallel jobs to use in params search
- **mode**: mode of meta train, may be: single_batch, batch or epoch, single_batch is more time efficient
- **double_meta_step**: boolean flag for double meta step (for supported NN)
- **meta_optimizer**: boolean flag for meta optimizer (Adam)
- **experiment_name**: str experiment name from the experiment_storage function
- **return**: dict of params for meta training

In [ ]:
params = er.meta_params(metadataset=dataset, model=model, tr_sub=tr_sub, trials=100,
                        experiment_name=experiment_name)

### Perform meta-training (pretrained models will be saved in experiment folder)
`
meta_exp(params: dict, model, target_sub: list, metadataset: MetaDataset, mode='single_batch',
             meta_optimizer=False, num_workers=1, experiment_name='experiment', all_subjects: list = None,
             baseline=True, early_stopping=0)
`
**meta_exp** - function for meta learning and baseline transfer learning
- **params** : dict, params for meta-learning from meta_params function
- **model** : model for which params search will be performed
- **target_sub** : list, target subjects for meta training (each will have a meta-trained model from which training this subject will be excluded)
- **metadataset** : MetaDataset, name of working dataset
- **mode**: 
- - 'single_batch' - only one batch per meta epoch size of **params[in_datasamples]**
- - 'batch' - all training data for each meta epoch in batches size of **params[in_datasamples]**
- - 'epoch' - all training data for each meta epoch in one batch,
-  **meta_optimizer** boolean flag for meta optimizer (Adam)
- **num_workers** : int, specifies number of threads for meta-learning experiments (usually 1 or 2), 
- **experiment_name** : str, experiment name from the experiment_storage function
- **all_subjects** : list or None, list of all subjects for meta-training if None all subjects from dataset will be used,
- **baseline** : boolean, prepare or not baseline model pretrained with transfer-learning
- **early_stopping** : int, meta-learning early stopping after **int** epochs without improvement if 0 - disabled

**For meta-training one neural network use function:**
`meta_train(params: dict, model, metadataset: MetaDataset, wal_sub, path, name: str = None,
               mode='single_batch', meta_optimizer=False, subjects: list = None, loging=True, baseline=True,
               early_stopping=0)`
- **wal_sub** : int, id of subject for validation
- **path** : str, where to save meta-trained model
- **name** : str, name for model weights file (if None will be **<wal_sub>_reptile.pkl**)

In [ ]:
pretraining_auc = er.meta_exp(params=params, model=model, target_sub=targ + [3], # here we add subj 3 to use him in parameters search for fine-tuning 
                              metadataset=dataset, mode='batch', meta_optimizer=False, num_workers=2,
                              experiment_name=experiment_name,
                              baseline= True)  # train or not similar network on same data with classic Transfer Learning
print('Pretraining completed. Mean auc for pretraining: ' + str(pretraining_auc))

### Find hyperparams for fine-tuning
- **lr**
- (**a**x + **b**) = Epochs, where x is number of datapoints for fine-tuning

In [ ]:
af_params = er.aftrain_params(metadataset=dataset, model=model, tst_subj=[3], trials=100, jobs=2,
                              experiment_name=experiment_name)

### Fine-tuning experiment (evaluation of performance on unseen subj)
(results will be stored in experiment folder)
`aftrain(target_sub, model, af_params, metadataset: MetaDataset, iterations=1, length=50, logging=False,
            experiment_name='experiment', last_layer=False)`
- **iterations**: int, number of fine-tuning experiments for each subject (allows you to get average statistics)
- **length** : int, max number of datapoints for fine-tuning
- **last_layer** : boolean, if used supported NN (freezes input layers)

In [ ]:
er.aftrain(target_sub=targ, model=model,
           af_params=af_params, metadataset=dataset,
           length=65, # max number of datapoints which will be used for fine-tuning
           iterations=5, # number of similar experiments with different random seed to get the mean stats
           experiment_name=experiment_name, last_layer=False)